## Вебинар 4. Домашнее задание

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('../data/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

# data_train = prefilter_items(data_train, item_features, take_n_popular=5000)
items = prefilter_items(data_train, item_features, take_n_popular=5000)
n_items_after = len(items)
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5000


In [6]:
# Заведем фиктивный item_id (если юзер не покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(items), 'item_id'] = 999999

C:\Users\nlitvak\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,201704,818981,819063,819112,819255,819304,819308,819330,819518,819594,...,15630096,15717057,15741861,15831255,15926712,15926775,15926844,15926886,15926887,15926927
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [9]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [10]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 2.42 s


-----

## Домашнее задание №4

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [11]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        
        # your_code
        
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        # Практически полностью реализовали на прошлом вебинаре

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [12]:
def get_similar_items_recommendation(data_train, result, model, n_recs=5):
    popularity = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)

    popularity = popularity[popularity['item_id'] != 999999]

    popularity = popularity.groupby('user_id').head(n_recs)

    popularity['similar_recommendation'] = popularity['item_id'].apply(
        lambda x: id_to_itemid[model.similar_items(itemid_to_id[x], N=2)[1][0]]
    )
    recommendation_similar_items = popularity.groupby('user_id')['similar_recommendation'].unique().reset_index()
    recommendation_similar_items.columns=['user_id', 'similar_recommendation']
    result = result.merge(recommendation_similar_items, on='user_id', how='left')
    return result

In [13]:
result = get_similar_items_recommendation(data_train, result, model)
result

,user_id,actual,similar_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[839818, 826597, 15926844, 5577022, 9297615]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1076875, 9526411, 960318, 1075979, 1133018]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[825541, 845208, 948650, 1023720, 5569845]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1019247, 834117, 1133018, 949836, 1084597]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1079067, 1106523, 5569845, 1110843, 8090532]"
...,...,...,...
2037,2496,[6534178],"[985999, 1096036, 5569845, 12757140, 865528]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[995785, 860776, 860528, 1125904, 970030]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1126899, 1058997, 860299, 9526411, 845262]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1126899, 5569327, 925626, 1062966, 973135]"


In [14]:
def get_similar_users(model, x):
    popularity = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)

    popularity = popularity[popularity['item_id'] != 999999]

    popularity = popularity.groupby('user_id').head(1)
    
    users = model.similar_users(userid_to_id[x], N=6)
    similar_users_buy = []
    for user in users:
        user_id_ = id_to_userid[user[0]]
        item = popularity.loc[popularity['user_id'] ==  user_id_,'item_id'].values
        if item.size > 0:
            similar_users_buy.append(item[0])
    return similar_users_buy[1:]

In [15]:
result['similar_users_buy'] = result['user_id'].apply(lambda x: get_similar_users(model, x))
result

,user_id,actual,similar_recommendation,similar_users_buy
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[839818, 826597, 15926844, 5577022, 9297615]","[1070820, 898847, 832442, 1057260, 1094924]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1076875, 9526411, 960318, 1075979, 1133018]","[1070820, 908531, 1044078, 8090521, 951590]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[825541, 845208, 948650, 1023720, 5569845]","[1053690, 1056212, 945652, 1051516, 1106523]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1019247, 834117, 1133018, 949836, 1084597]","[947798, 819330, 1085604, 962266, 951590]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1079067, 1106523, 5569845, 1110843, 8090532]","[1106523, 916990, 1126899, 868764, 961554]"
...,...,...,...,...
2037,2496,[6534178],"[985999, 1096036, 5569845, 12757140, 865528]","[844165, 1133018, 844165, 951590, 1106523]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[995785, 860776, 860528, 1125904, 970030]","[1106523, 1070820, 1101010, 873964, 833351]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1126899, 1058997, 860299, 9526411, 845262]","[995965, 862349, 9837076, 951590, 916990]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1126899, 5569327, 925626, 1062966, 973135]","[907631, 1092026, 1137346, 844165, 907631]"
